In [1]:
import os 
import openai
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text

In [5]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
df = pd.read_csv('sales_data_sample.csv')

In [7]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [8]:
temp_db = create_engine('sqlite:///:memory:',echo=True)

In [9]:
data = df.to_sql(name='Sales',con=temp_db)

2023-03-06 15:54:08,586 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 15:54:08,587 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-03-06 15:54:08,589 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 15:54:08,590 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-03-06 15:54:08,591 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 15:54:08,593 INFO sqlalchemy.engine.Engine ROLLBACK
2023-03-06 15:54:08,598 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 15:54:08,600 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	

In [10]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT sum(SALES) FROM Sales"))

2023-03-06 15:54:12,999 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 15:54:13,000 INFO sqlalchemy.engine.Engine SELECT sum(SALES) FROM Sales
2023-03-06 15:54:13,001 INFO sqlalchemy.engine.Engine [generated in 0.00186s] ()
2023-03-06 15:54:13,003 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
result.all()

[(10032628.85000001,)]

In [12]:
def create_table_definition(df):
    prompt = """### sqlite SQL table . with it properties:
    #
    # Sales({})
    #
    """.format(",".join(str(col) for col in df.columns))
    
    return prompt

In [13]:
print(create_table_definition(df))

### sqlite SQL table . with it properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    


In [14]:
def prompt_input():
    nlp_text = input("Enter the info you want : ")
    return nlp_text

In [15]:
prompt_input()

Enter the info you want : dadd


'dadd'

In [16]:
def compin_prompt(df,query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition+query_init_string

In [23]:
nlp_text = prompt_input()
compin_prompt(df,nlp_text)

Enter the info you want : return the sum of SALES per POSTALCODE


'### sqlite SQL table . with it properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    #\n    ### A query to answer: return the sum of SALES per POSTALCODE\nSELECT'

In [24]:
compin_prompt(df,nlp_text)

'### sqlite SQL table . with it properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    #\n    ### A query to answer: return the sum of SALES per POSTALCODE\nSELECT'

In [25]:
response = openai.Completion.create(
    model = 'code-davinci-002',
    prompt = compin_prompt(df,nlp_text),
    temperature = 0,
    max_tokens = 150,
    top_p=1.0,
    frequency_penalty = 0,
    presence_penalty =  0,
    stop = ["#",";"]
    
)

In [26]:
print(response['choices'][0]['text'])

 POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE
    


In [27]:
def handle_response(response):
    qury = response['choices'][0]['text']
    if qury.startswith(" "):
        qury= "SELECT"+qury
    return qury
        

In [28]:
handle_response(response)

'SELECT POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE\n    '

In [29]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-03-06 16:05:09,031 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 16:05:09,032 INFO sqlalchemy.engine.Engine SELECT POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE
    
2023-03-06 16:05:09,034 INFO sqlalchemy.engine.Engine [generated in 0.00299s] ()
2023-03-06 16:05:09,039 INFO sqlalchemy.engine.Engine ROLLBACK


In [30]:
result.all()

[(None, 272407.14),
 ('10022', 560787.7699999998),
 ('10100', 94117.26000000002),
 ('106-0032', 120562.73999999996),
 ('1203', 117713.55999999998),
 ('1227 MM', 94015.73),
 ('13008', 74936.14),
 ('1734', 145041.6),
 ('2', 57756.43),
 ('2060', 153996.13000000003),
 ('2067', 151570.98000000004),
 ('21240', 111250.37999999996),
 ('24067', 85555.98999999998),
 ('24100', 137955.72000000003),
 ('28023', 170257.33000000005),
 ('28034', 912294.1100000002),
 ('3004', 200995.40999999997),
 ('31000', 70488.44),
 ('3150', 64591.46000000001),
 ('4101', 59469.11999999999),
 ('4110', 116599.19),
 ('41101', 54723.62),
 ('42100', 142601.33000000002),
 ('44000', 204304.86),
 ('5020', 149798.63),
 ('50553', 207874.86),
 ('50739', 100306.58),
 ('51003', 154069.65999999997),
 ('51100', 135042.94),
 ('51247', 139243.99999999994),
 ('530-0003', 67605.07),
 ('58339', 165255.20000000004),
 ('59000', 69052.41),
 ('60528', 85171.58999999998),
 ('62005', 131685.30000000002),
 ('67000', 80438.48),
 ('69004', 14287